<a href="https://colab.research.google.com/github/edson-depaula/DIO_BairesDev_Bootcamp/blob/main/Projeto_Detec%C3%A7%C3%A3o_Facial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================================================
# PRÉ-REQUISITOS E INSTALAÇÕES
# =============================================================================

# Instalar bibliotecas necessárias
!pip install scikit-learn joblib keras-facenet mtcnn
!pip install --upgrade lz4 # A linha que foi adicionada para garantir

# Montar o Google Drive para acessar as imagens de treinamento
from google.colab import drive
drive.mount('/content/drive')

# =============================================================================
# IMPORTS E FUNÇÕES AUXILIARES
# =============================================================================

import os
import cv2
import numpy as np
import joblib
from keras_facenet import FaceNet
from mtcnn import MTCNN
from google.colab import files
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, Image
from base64 import b64decode
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics.pairwise import cosine_similarity

# Definir o caminho para o seu projeto no Google Drive
drive_path = '/content/drive/MyDrive/Projeto Reconhecimento Facial'
dataset_path = drive_path

def get_face_embedding(model, face_array):
    """Gera um embedding de 128 dimensões para uma face."""
    face_array = (face_array - face_array.mean()) / face_array.std()
    face_expandida = np.expand_dims(face_array, axis=0)
    embedding = model.model.predict(face_expandida)
    return embedding[0]

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto() {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capturar';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      return new Promise((resolve) => {
        capture.onclick = () => {
          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          resolve(canvas.toDataURL('image/jpeg', %f));
        };
      });
    }
    '''.replace('%f', str(quality)))
  display(js)
  data = eval_js('takePhoto()')
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Adicione esta importação no topo do seu script, se ainda não tiver
from sklearn.metrics.pairwise import cosine_similarity

# -----------------------------------------------------------------------------
# Nova versão da função processar_e_mostrar_imagem
# -----------------------------------------------------------------------------

def processar_e_mostrar_imagem(frame, svm_classifier, label_encoder, model_facenet, detector_de_faces):
    """Processa uma imagem e exibe o resultado com um limite de confiança."""

    # Detecção de faces com MTCNN
    faces_detectadas = detector_de_faces.detect_faces(frame)
    frame_copy = frame.copy() # Cria uma cópia para evitar modificações indesejadas

    # Definir o limite de confiança
    threshold_confianca = 0.75  # Este valor pode ser ajustado (de 0.0 a 1.0)

    for face_info in faces_detectadas:
        x, y, w, h = face_info['box']
        face_recortada = frame_copy[y:y+h, x:x+w]

        try:
            face_redimensionada = cv2.resize(face_recortada, (160, 160))

            # Gerar o embedding da face atual
            embedding_da_face = get_face_embedding(model_facenet, face_redimensionada)
            embedding_da_face = np.expand_dims(embedding_da_face, axis=0)

            # Fazer a predição e obter as probabilidades
            predicoes_prob = svm_classifier.predict_proba(embedding_da_face)[0]
            label_predito = svm_classifier.predict(embedding_da_face)[0]

            # Achar a maior probabilidade para a predição
            probabilidade_max = np.max(predicoes_prob)

            if probabilidade_max > threshold_confianca:
                nome_da_pessoa = label_encoder.inverse_transform([label_predito])[0]
            else:
                nome_da_pessoa = "Desconhecido"

            # Desenhar o retângulo e o nome
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, nome_da_pessoa, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        except Exception as e:
            print(f"Erro ao processar a face: {e}")
            pass

    cv2_imshow(frame)

# =============================================================================
# PARTE 1 e 2: EXTRAÇÃO, AUMENTO DE DADOS E TREINAMENTO (Versão estável)
# =============================================================================

print("Iniciando a extração de embeddings e o aumento de dados...")
model_facenet = FaceNet()
detector_de_faces = MTCNN()
embeddings = []
nomes = []

# Aumentar a quantidade de dados de treinamento com as fotos existentes
for nome_pessoa in os.listdir(dataset_path):
    caminho_pessoa = os.path.join(dataset_path, nome_pessoa)
    if not os.path.isdir(caminho_pessoa): continue

    print(f"Processando e aumentando fotos de: {nome_pessoa}")

    # Carregar todas as fotos da pessoa
    for nome_arquivo in os.listdir(caminho_pessoa):
        caminho_imagem = os.path.join(caminho_pessoa, nome_arquivo)
        imagem = cv2.imread(caminho_imagem)
        if imagem is None: continue

        # Encontrar e processar a face na imagem
        faces_detectadas = detector_de_faces.detect_faces(imagem)
        if len(faces_detectadas) > 0:
            face_info = faces_detectadas[0] # Pega a primeira face
            x, y, w, h = face_info['box']
            face_recortada = imagem[y:y+h, x:x+w]

            try:
                face_redimensionada = cv2.resize(face_recortada, (160, 160))
                embedding = get_face_embedding(model_facenet, face_redimensionada)
                embeddings.append(embedding)
                nomes.append(nome_pessoa)
            except Exception as e:
                print(f"Não foi possível processar a imagem {nome_arquivo}: {e}")

embeddings = np.array(embeddings)
nomes = np.array(nomes)

print(f"Total de embeddings extraídos: {len(embeddings)}")
print("Extração de embeddings concluída.")

# Treinamento do classificador
print("\nIniciando o treinamento do classificador...")
label_encoder = LabelEncoder()
labels_numericas = label_encoder.fit_transform(nomes)
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(embeddings, labels_numericas)
joblib.dump(svm_classifier, 'svm_classifier.joblib')
joblib.dump(label_encoder, 'label_encoder.joblib')
print("Treinamento do classificador concluído e modelos salvos.")

# =============================================================================
# PARTE 3: RECONHECIMENTO POR IMAGEM (CAPTURA OU UPLOAD)
# =============================================================================

print("\nModelos carregados. Escolha uma opção abaixo para o reconhecimento.")

# Carregar os modelos de predição
svm_classifier = joblib.load('svm_classifier.joblib')
label_encoder = joblib.load('label_encoder.joblib')
model_facenet = FaceNet()
detector_de_faces = MTCNN()

# --- Opção 1: Capturar foto com a webcam ---
print("\nOpção 1: Clique no botão 'Capturar' abaixo para tirar uma foto com sua webcam.")
try:
    filename = take_photo()
    print(f'Foto salva em "{filename}"')

    img = cv2.imread(filename)
    processar_e_mostrar_imagem(img, svm_classifier, label_encoder, model_facenet, detector_de_faces)

except Exception as err:
    print(f"Erro na Opção 1: {err}")

# --- Opção 2: Fazer upload de uma imagem ---
print("\nOpção 2: Clique no botão 'Choose Files' para fazer upload de uma imagem.")
try:
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'Uploaded file "{filename}"')
        img = cv2.imread(filename)
        processar_e_mostrar_imagem(img, svm_classifier, label_encoder, model_facenet, detector_de_faces)

except Exception as err:
    print(f"Erro na Opção 2: {err}")

In [ ]:
# =============================================================================
# BLOCO DE RECONHECIMENTO POR WEBCAM (APÓS O TREINAMENTO)
# =============================================================================

import os
import cv2
import numpy as np
import joblib
from keras_facenet import FaceNet
from mtcnn import MTCNN
from google.colab import files, drive
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript
from google.colab.output import eval_js  # A linha que faltava
from base64 import b64decode
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# --- Funções Auxiliares Necessárias ---

# Certifique-se de que o Drive está montado para carregar os modelos
drive.mount('/content/drive', force_remount=True)

def get_face_embedding(model, face_array):
    """Gera um embedding de 128 dimensões para uma face."""
    face_array = (face_array - face_array.mean()) / face_array.std()
    face_expandida = np.expand_dims(face_array, axis=0)
    embedding = model.model.predict(face_expandida)
    return embedding[0]

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto() {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capturar';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      return new Promise((resolve) => {
        capture.onclick = () => {
          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          resolve(canvas.toDataURL('image/jpeg', %f));
        };
      });
    }
    '''.replace('%f', str(quality)))
  display(js)
  data = eval_js('takePhoto()')
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

def processar_e_mostrar_imagem(frame, svm_classifier, label_encoder, model_facenet, detector_de_faces):
    """Processa uma imagem e exibe o resultado com um limite de confiança."""

    faces_detectadas = detector_de_faces.detect_faces(frame)
    frame_copy = frame.copy()

    threshold_confianca = 0.75

    for face_info in faces_detectadas:
        x, y, w, h = face_info['box']
        face_recortada = frame_copy[y:y+h, x:x+w]

        try:
            face_redimensionada = cv2.resize(face_recortada, (160, 160))

            embedding_da_face = get_face_embedding(model_facenet, face_redimensionada)
            embedding_da_face = np.expand_dims(embedding_da_face, axis=0)

            predicoes_prob = svm_classifier.predict_proba(embedding_da_face)[0]
            label_predito = svm_classifier.predict(embedding_da_face)[0]

            probabilidade_max = np.max(predicoes_prob)

            if probabilidade_max > threshold_confianca:
                nome_da_pessoa = label_encoder.inverse_transform([label_predito])[0]
            else:
                nome_da_pessoa = "Desconhecido"

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, nome_da_pessoa, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        except Exception as e:
            print(f"Erro ao processar a face: {e}")
            pass

    cv2_imshow(frame)


# --- Execução do Reconhecimento ---

print("\nCarregando modelos para reconhecimento...")

# Carregar os modelos de predição
svm_classifier = joblib.load('svm_classifier.joblib')
label_encoder = joblib.load('label_encoder.joblib')
model_facenet = FaceNet()
detector_de_faces = MTCNN()

print("\nModelos carregados. Clique no botão para tirar uma foto.")
try:
    filename = take_photo()
    print(f'Foto salva em "{filename}"')

    img = cv2.imread(filename)
    processar_e_mostrar_imagem(img, svm_classifier, label_encoder, model_facenet, detector_de_faces)

except Exception as err:
    print(f"Erro ao capturar ou processar a foto: {err}")

In [ ]:
# =============================================================================
# BLOCO DE RECONHECIMENTO POR UPLOAD DE FOTO (APÓS O TREINAMENTO)
# =============================================================================

import os
import cv2
import numpy as np
import joblib
from keras_facenet import FaceNet
from mtcnn import MTCNN
from google.colab import files, drive
from google.colab.patches import cv2_imshow
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# --- Funções Auxiliares Necessárias ---

# Certifique-se de que o Drive está montado para carregar os modelos
drive.mount('/content/drive', force_remount=True)

def get_face_embedding(model, face_array):
    """Gera um embedding de 128 dimensões para uma face."""
    face_array = (face_array - face_array.mean()) / face_array.std()
    face_expandida = np.expand_dims(face_array, axis=0)
    embedding = model.model.predict(face_expandida)
    return embedding[0]

def processar_e_mostrar_imagem(frame, svm_classifier, label_encoder, model_facenet, detector_de_faces):
    """Processa uma imagem e exibe o resultado com um limite de confiança."""

    faces_detectadas = detector_de_faces.detect_faces(frame)
    frame_copy = frame.copy()

    # Utilize o threshold de confiança que você ajustou
    threshold_confianca = 0.75

    for face_info in faces_detectadas:
        x, y, w, h = face_info['box']
        face_recortada = frame_copy[y:y+h, x:x+w]

        try:
            face_redimensionada = cv2.resize(face_recortada, (160, 160))

            embedding_da_face = get_face_embedding(model_facenet, face_redimensionada)
            embedding_da_face = np.expand_dims(embedding_da_face, axis=0)

            predicoes_prob = svm_classifier.predict_proba(embedding_da_face)[0]
            label_predito = svm_classifier.predict(embedding_da_face)[0]

            probabilidade_max = np.max(predicoes_prob)

            if probabilidade_max > threshold_confianca:
                nome_da_pessoa = label_encoder.inverse_transform([label_predito])[0]
            else:
                nome_da_pessoa = "Desconhecido"

            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, nome_da_pessoa, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        except Exception as e:
            print(f"Erro ao processar a face: {e}")
            pass

    cv2_imshow(frame)


# --- Execução do Reconhecimento ---

print("\nCarregando modelos para reconhecimento...")

# Carregar os modelos de predição
svm_classifier = joblib.load('svm_classifier.joblib')
label_encoder = joblib.load('label_encoder.joblib')
model_facenet = FaceNet()
detector_de_faces = MTCNN()

print("\nModelos carregados. Clique no botão 'Choose Files' para fazer upload de uma imagem.")
try:
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'Uploaded file "{filename}"')
        img = cv2.imread(filename)
        processar_e_mostrar_imagem(img, svm_classifier, label_encoder, model_facenet, detector_de_faces)

except Exception as err:
    print(f"Erro ao fazer upload ou processar a foto: {err}")